<a href="https://colab.research.google.com/github/jhwaffles/cgm_peak_detection/blob/main/peak_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from peak_detection_pipeline import run_event_metrics_pipeline
from datetime import datetime,time,timedelta
import matplotlib.pyplot as plt

In [ ]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
print()
raw_data_dir = os.path.join(parent_dir, 'example_data')
user='user2'
file_name=user+'.csv'
cgm_file_path = os.path.join(raw_data_dir, file_name)

df = pd.read_csv(cgm_file_path, skiprows=1)

#start_time = pd.to_datetime('2024-09-09 12:00')
#end_time = pd.to_datetime('2024-09-21 00:00')

start_time = pd.to_datetime('2022-07-04 12:00')
end_time = pd.to_datetime('2022-07-14 00:00')


df['timestamp'] = pd.to_datetime(df['Device Timestamp'], format="%m/%d/%Y %H:%M", errors='coerce')
df['glucose'] = pd.to_numeric(df['Historic Glucose mg/dL'], errors='coerce')
df['time_since_midnight'] = df['timestamp'] - df['timestamp'].dt.normalize()
df['hours_since_midnight'] = df['time_since_midnight'].dt.total_seconds() / 3600
df['date'] = df['timestamp'].dt.date

df=df[df['Record Type']==0]
df=df.dropna(subset=['timestamp']).sort_values('timestamp').reset_index(drop=True)

start_dt = datetime.combine(start_time, time.min)
end_dt = datetime.combine(end_time, time.max)
df_filtered=df[(df['timestamp'] >= start_dt) & (df['timestamp'] <= end_dt)]
df_filtered

,Device,Serial Number,Device Timestamp,Record Type,Historic Glucose mg/dL,Scan Glucose mg/dL,Non-numeric Rapid-Acting Insulin,Rapid-Acting Insulin (units),Non-numeric Food,Carbohydrates (grams),...,Strip Glucose mg/dL,Ketone mmol/L,Meal Insulin (units),Correction Insulin (units),User Change Insulin (units),timestamp,glucose,time_since_midnight,hours_since_midnight,date
0,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 12:27,0,96.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-04 12:27:00,96.0,0 days 12:27:00,12.450000,2022-07-04
1,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 12:42,0,94.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-04 12:42:00,94.0,0 days 12:42:00,12.700000,2022-07-04
2,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 12:57,0,98.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-04 12:57:00,98.0,0 days 12:57:00,12.950000,2022-07-04
3,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 13:12,0,102.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-04 13:12:00,102.0,0 days 13:12:00,13.200000,2022-07-04
4,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 13:27,0,101.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-04 13:27:00,101.0,0 days 13:27:00,13.450000,2022-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/14/2022 21:50,0,103.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-14 21:50:00,103.0,0 days 21:50:00,21.833333,2022-07-14
979,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/14/2022 22:05,0,116.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-14 22:05:00,116.0,0 days 22:05:00,22.083333,2022-07-14
980,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/14/2022 22:20,0,141.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-14 22:20:00,141.0,0 days 22:20:00,22.333333,2022-07-14
981,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/14/2022 22:35,0,166.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-07-14 22:35:00,166.0,0 days 22:35:00,22.583333,2022-07-14


In [ ]:
slope_threshold=0.45
min_glucose=85

try:
    #calculate rolling derivative of signal.
    df=df_filtered.copy()
    times_np = np.array(df['timestamp'])
    glucose_np = np.array(df['glucose'])
    df['slope'] = df['glucose'].diff(periods=2) / 30
    #find anything where slope is positive AND glucose level is greater than 100 mg/dL. extract these as start times as events.
    mask = (df['slope'] >= slope_threshold) & (df['glucose'] >= min_glucose) #1 if yes, 0 if no.
    candidate_peaks = df[mask].copy()

    events = []
    if not candidate_peaks.empty:
        last_event_time = None
        for idx, row in candidate_peaks.iterrows():

            start_idx = max(0, df.index.get_loc(idx) - 2) #when threshold cross, mark start of event at 2 rows back (30 minutes) due to diff calculation.
            actual_start_row = df.iloc[start_idx].copy()
            if last_event_time is None or (row['timestamp'] - last_event_time).total_seconds() / 60 > 90:  #is within 90 minutes, make it same event.
                events.append(actual_start_row)
                last_event_time = actual_start_row['timestamp']

    events_df = pd.DataFrame(events)

except Exception as e:
    print("[DEBUG] extract_events ERROR:", e)

events_df

,Device,Serial Number,Device Timestamp,Record Type,Historic Glucose mg/dL,Scan Glucose mg/dL,Non-numeric Rapid-Acting Insulin,Rapid-Acting Insulin (units),Non-numeric Food,Carbohydrates (grams),...,Ketone mmol/L,Meal Insulin (units),Correction Insulin (units),User Change Insulin (units),timestamp,glucose,time_since_midnight,hours_since_midnight,date,slope
4,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 13:27,0,101.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-04 13:27:00,101.0,0 days 13:27:00,13.450000,2022-07-04,0.100000
9,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 14:42,0,121.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-04 14:42:00,121.0,0 days 14:42:00,14.700000,2022-07-04,-1.233333
31,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/4/2022 20:13,0,90.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-04 20:13:00,90.0,0 days 20:13:00,20.216667,2022-07-04,-0.233333
55,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/5/2022 3:00,0,76.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-05 03:00:00,76.0,0 days 03:00:00,3.000000,2022-07-05,0.166667
78,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/5/2022 8:46,0,76.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-05 08:46:00,76.0,0 days 08:46:00,8.766667,2022-07-05,-0.500000
87,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/5/2022 11:01,0,101.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-05 11:01:00,101.0,0 days 11:01:00,11.016667,2022-07-05,0.066667
103,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/5/2022 15:02,0,95.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-05 15:02:00,95.0,0 days 15:02:00,15.033333,2022-07-05,-0.266667
137,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/5/2022 23:32,0,99.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-05 23:32:00,99.0,0 days 23:32:00,23.533333,2022-07-05,-0.133333
171,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/6/2022 8:04,0,94.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-06 08:04:00,94.0,0 days 08:04:00,8.066667,2022-07-06,0.300000
178,FreeStyle LibreLink,377ff7b3-4706-4d1d-a27a-0c8dda28b976,7/6/2022 9:49,0,106.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-07-06 09:49:00,106.0,0 days 09:49:00,9.816667,2022-07-06,0.166667


In [ ]:
def create_event_windows(df_events, window_duration=pd.Timedelta(hours=2)):
    df_events = df_events.sort_values(by='timestamp').reset_index(drop=True)
    df_events['timestamp'] = pd.to_datetime(df_events['timestamp'])

    event_windows = []

    for i in range(len(df_events)):
        event_time = df_events.loc[i, 'timestamp']
        note = df_events.loc[i].get('Notes', pd.NA)

        window_start = event_time
        # Default end: event_time + 2 hours
        default_end = event_time + window_duration

        # If there's a next event, use that as the cap
        if i < len(df_events) - 1:
            next_event_time = df_events.loc[i + 1, 'timestamp']
            window_end = min(default_end, next_event_time)
        else:
            window_end = default_end

        event_windows.append({
            'event_time': event_time,
            'event_note': note,
            'window_start': window_start,
            'window_end': window_end
        })

    return pd.DataFrame(event_windows)

In [ ]:
event_windows=create_event_windows(events_df)
event_windows

,event_time,event_note,window_start,window_end
0,2022-07-04 13:27:00,NaN,2022-07-04 13:27:00,2022-07-04 14:42:00
1,2022-07-04 14:42:00,NaN,2022-07-04 14:42:00,2022-07-04 16:42:00
2,2022-07-04 20:13:00,NaN,2022-07-04 20:13:00,2022-07-04 22:13:00
3,2022-07-05 03:00:00,NaN,2022-07-05 03:00:00,2022-07-05 05:00:00
4,2022-07-05 08:46:00,NaN,2022-07-05 08:46:00,2022-07-05 10:46:00
5,2022-07-05 11:01:00,NaN,2022-07-05 11:01:00,2022-07-05 13:01:00
6,2022-07-05 15:02:00,NaN,2022-07-05 15:02:00,2022-07-05 17:02:00
7,2022-07-05 23:32:00,NaN,2022-07-05 23:32:00,2022-07-06 01:32:00
8,2022-07-06 08:04:00,NaN,2022-07-06 08:04:00,2022-07-06 09:49:00
9,2022-07-06 09:49:00,NaN,2022-07-06 09:49:00,2022-07-06 11:49:00


In [ ]:
def get_time_category(dt):
    hour = dt.hour
    if 6 <= hour < 10:
        return 0  # morning
    elif 10 <= hour < 14:
        return 1  # noon
    elif 14 <= hour < 18:
        return 2  # afternoon
    elif 18 <= hour < 22:
        return 3  # evening
    else:
        return 4 # late night

def compute_window_metrics(df_cgm, start, end):
    """
    Given a CGM dataframe and a time window, return glucose metrics.
    Assumes df_cgm['timestamp'] and df_cgm['glucose'] exist.
    """
    # Filter CGM data in the window
    mask = (df_cgm['timestamp'] >= start) & (df_cgm['timestamp'] <= end)
    window_data = df_cgm.loc[mask].copy()

    if window_data.empty:
        return {
            'glucose_max': np.nan,
            'glucose_min': np.nan,
            'glucose_auc': np.nan,
            'glucose_rate_rise': np.nan,
            'glucose_rate_fall': np.nan
        }

    # Sort just in case
    window_data = window_data.sort_values(by='timestamp')

    # Calculate time deltas in hours for AUC
    times = (window_data['timestamp'] - window_data['timestamp'].iloc[0]).dt.total_seconds() / 60.0/ 60.0
    glucose = window_data['glucose'].values

    # Basic metrics
    g_max = glucose.max()
    g_min = glucose.min()

    glucose_start=glucose[0]
    adjusted_glucose =np.maximum(glucose - glucose_start, 0 )
    auc = np.trapezoid(adjusted_glucose, times)

    # Rate of change. finds sliding window slopes for 3 consecutive points. Takes the max.
    glucose_diff = np.diff(glucose)
    time_diff = np.diff(times)
    slopes = np.divide(glucose_diff, time_diff, out=np.zeros_like(glucose_diff), where=time_diff!=0)
    rate_


    rate_rise = np.max(slopes) if len(slopes) > 0 else np.nan
    rate_fall = np.min(slopes) if len(slopes) > 0 else np.nan
    print('for event starting at: {}'.format(start))
    print('glucose_diff: {}'.format(glucose_diff))
    print('time_diff: {}'.format(time_diff))

    return {
        'glucose_max': g_max,
        'glucose_min': g_min,
        'glucose_auc': auc,
        'glucose_rate_rise': rate_rise,
        'glucose_rate_fall': rate_fall
    }

def compute_metrics_for_all_windows(df_event_windows, df_cgm_data, df_events):
    metrics = []
    for _, row in df_event_windows.iterrows():
        start = row['window_start']
        end = row['window_end']
        event_time = row['event_time']
        event_info = row.to_dict()

        # Add: meal_time_category
        #meal_time_category = get_time_category(event_time)
        meal_time_category = get_time_category(event_time)

        m = compute_window_metrics(df_cgm_data, start, end)
        metrics.append({
            **event_info,
            **m,
            **m,
            'meal_time_category': meal_time_category
            })

    return pd.DataFrame(metrics)

In [ ]:
df_event_metrics = compute_metrics_for_all_windows(event_windows, df_filtered, events_df)
df_event_metrics

for event starting at: 2022-07-04 13:27:00
glucose_diff: [  7.  20.  30. -13. -24.]
time_diff: [0.25       0.25       0.25       0.26666667 0.23333333]
for event starting at: 2022-07-04 14:42:00
glucose_diff: [-10.  24. -16. -17.  -3.   1.   5.]
time_diff: [0.25       0.26666667 0.25       0.25       0.26666667 0.25
 0.25      ]
for event starting at: 2022-07-04 20:13:00
glucose_diff: [  8.  14.  39.  49.  17. -18. -26.]
time_diff: [0.25       0.25       0.25       0.25       0.26666667 0.25
 0.25      ]
for event starting at: 2022-07-05 03:00:00
glucose_diff: [ 6. 10.  0. -1.  2.  0. -2.  0.]
time_diff: [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
for event starting at: 2022-07-05 08:46:00
glucose_diff: [ 7. 11.  1. -5. -1.  7.  3.  0.]
time_diff: [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
for event starting at: 2022-07-05 11:01:00
glucose_diff: [ 6.  8.  4. -6. -5. -1.  1.  2.]
time_diff: [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
for event starting at: 2022-07-05 15:02:00
glucose_diff: [

,event_time,event_note,window_start,window_end,glucose_max,glucose_min,glucose_auc,glucose_rate_rise,glucose_rate_fall,meal_time_category
0,2022-07-04 13:27:00,NaT,2022-07-04 13:27:00,2022-07-04 14:42:00,158.0,101.0,36.558333,120.00000,-102.857143,1
1,2022-07-04 14:42:00,NaT,2022-07-04 14:42:00,2022-07-04 16:42:00,135.0,99.0,3.616667,90.00000,-68.000000,2
2,2022-07-04 20:13:00,NaT,2022-07-04 20:13:00,2022-07-04 22:13:00,217.0,90.0,121.600000,196.00000,-104.000000,3
3,2022-07-05 03:00:00,NaT,2022-07-05 03:00:00,2022-07-05 05:00:00,93.0,76.0,27.375000,40.00000,-8.000000,4
4,2022-07-05 08:46:00,NaT,2022-07-05 08:46:00,2022-07-05 10:46:00,99.0,76.0,31.375000,44.00000,-20.000000,0
5,2022-07-05 11:01:00,NaT,2022-07-05 11:01:00,2022-07-05 13:01:00,119.0,101.0,18.625000,32.00000,-24.000000,1
6,2022-07-05 15:02:00,NaT,2022-07-05 15:02:00,2022-07-05 17:02:00,156.0,95.0,89.250000,116.00000,-40.000000,2
7,2022-07-05 23:32:00,NaT,2022-07-05 23:32:00,2022-07-06 01:32:00,131.0,99.0,38.075000,64.00000,-24.705882,4
8,2022-07-06 08:04:00,NaT,2022-07-06 08:04:00,2022-07-06 09:49:00,108.0,94.0,13.250000,32.00000,-28.000000,0
9,2022-07-06 09:49:00,NaT,2022-07-06 09:49:00,2022-07-06 11:49:00,147.0,99.0,24.250000,88.00000,-96.000000,0


In [ ]:
def create_event_plot(df_day, df_events, day_str, show_events=False):
    fig = go.Figure()

    # Standard Glucose Trace
    fig.add_trace(go.Scatter(
        x=df_day['timestamp'],
        y=df_day['glucose'],
        mode='lines+markers',
        name='Glucose',
        line=dict(color='#1f77b4'),
        hovertemplate='Time: %{x}<br>Glucose: %{y} mg/dL<extra></extra>'
    ))

    if show_events:
        # Standardize day_str to string for comparison
        target_date = str(day_str)

        for _, row in df_events.iterrows():
            # Robust date check
            event_date = row['window_start'].strftime('%Y-%m-%d')
            if event_date != target_date:
                continue

            # Ensure timestamps are comparable (both naive or both aware)
            # This mask creates the highlighted area
            mask = (df_day['timestamp'] >= row['window_start']) & \
                   (df_day['timestamp'] <= row['window_end'])

            segment = df_day.loc[mask].sort_values('timestamp')

            if len(segment) < 2:
                print(f"[DEBUG] Skipping event {row.get('event_note')} - segment too small ({len(segment)} rows)")
                continue

            baseline_val = segment['glucose'].iloc[0]
            # Focus on the 'rise'
            above = segment[segment['glucose'] >= baseline_val]

            if len(above) >= 2:
                fill_x = list(above['timestamp']) + list(above['timestamp'])[::-1]
                fill_y = list(above['glucose']) + [baseline_val] * len(above)

                # Plotly sometimes struggles with numpy types in customdata
                # Convert to standard Python types for stability
                cdat = [[
                    str(row.get('event_note', 'Event')),
                    row['window_start'].strftime("%H:%M"),
                    row['glucose_max'],
                    row['glucose_auc']
                ]] * len(fill_x)

                fig.add_trace(go.Scatter(
                    x=fill_x,
                    y=fill_y,
                    fill='toself',
                    mode='lines',
                    fillcolor='rgba(255, 165, 0, 0.4)',
                    line=dict(color='rgba(255, 165, 0, 0)'),
                    name=str(row.get('event_note', 'Event')),
                    customdata=cdat,
                    hovertemplate=(
                        "<b>%{customdata[0]}</b><br>" +
                        "Start: %{customdata[1]}<br>" +
                        "Peak: %{customdata[2]} mg/dL<br>" +
                        "AUC: %{customdata[3]:.1f}<extra></extra>"
                    ),
                    showlegend=False
                ))

    fig.update_layout(
        title=f'Glucose Events for {day_str}',
        xaxis_title='Time',
        yaxis_title='Glucose (mg/dL)',
        template='plotly_white'
    )
    return fig

In [ ]:
day_str=df_filtered['date'].unique()[0]
df_day = df[df['date'] == day_str]
#df_day.head()
create_event_plot(df_day, df_event_metrics, day_str, show_events=True)


In [ ]:
df_event_metrics.to_csv(user+'_event_features.csv')

annotate.
combine sets.
plot with true and false as different colors. get toggle to work.
then perform classification. need to stratify by days?
show confusion matrix and outputs. decision boundaries?